In [1]:
import sys
sys.path.insert(0, ".")

from pathlib import Path

import singleton_helpers

# Counter

Every time the singleton is spent, the count is incresed by one.


key/values = ["Counter", "A"]

solution has value



In [2]:
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)

from cdv.test import Network, Wallet
from cdv.util.load_clvm import load_clvm
from clvm_tools.clvmc import compile_clvm_text

network: Network = await Network.create()
await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice) # alice has 2000_000_000_000 mojos

print(f'alice: {alice.balance()}')
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

counter_clsp = Path('singleton_counter.clsp').read_text()

counter_puzzle = Program(
    compile_clvm_text(counter_clsp, search_paths=["../include"])
)

inner_puzzle = singleton_top_layer.adapt_inner_to_singleton(counter_puzzle)

standard_coin_message, coin_spends = singleton_helpers.get_create_singleton_coin_spends(
                    standard_txn_coin,
                    standard_txn_puzzle,
                    odd_amount = 1023, 
                    inner_puzzle = inner_puzzle,
                    keys_values = [("Counter", "A")]
              )
standard_txn_coin_spend = coin_spends[0]
launcher_coin_spend = coin_spends[1]
print("\nstarting coin spend:")
singleton_helpers.print_json(standard_txn_coin_spend.to_json_dict())
print("\nlauncher coin spend:")
singleton_helpers.print_json(launcher_coin_spend.to_json_dict())
print(f'\nlauncher id: {launcher_coin_spend.coin.name()}')
print(f'\nstandard_coin_message: {standard_coin_message.hex()}')


alice: 2000000000000

starting coin spend:
{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da

In [4]:
from blspy import AugSchemeMPL, PrivateKey
from chia.consensus.default_constants import DEFAULT_CONSTANTS

synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)

signature = AugSchemeMPL.sign(synthetic_sk,standard_coin_message)

In [5]:
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    coin_spends,
    signature
)
result = await network.push_tx(spend_bundle)
result

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1749999998977),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>, amount=1023),
  Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: 98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c06

In [6]:
launcher_id = launcher_coin_spend.coin.name()
singleton_struct = (
        singleton_top_layer.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer.SINGLETON_LAUNCHER_HASH)
    )

singleton_puzzle = singleton_top_layer.SINGLETON_MOD.curry(
            singleton_struct,
            inner_puzzle,
    )

singleton_coin_records = await network.sim_client.get_coin_records_by_parent_ids([launcher_id])
singleton_coin = singleton_coin_records[0].coin

lineage_proof = singleton_top_layer.lineage_proof_for_coinsol(launcher_coin_spend)

inner_solution = Program.to([
    singleton_puzzle.get_tree_hash(), 
    singleton_coin.amount, 
    inner_puzzle.get_tree_hash(), 
    1
])

singleton_coin_spend = singleton_helpers.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
    )

print(singleton_coin)
print(lineage_proof)
print(inner_solution)

singleton_helpers.print_json(singleton_coin_spend.to_json_dict())

{'amount': 1023,
 'parent_coin_info': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570',
 'puzzle_hash': '0x98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a'}
{'amount': 1023,
 'inner_puzzle_hash': None,
 'parent_name': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba'}
ffa098a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50aff8203ffffa01194ba575907e87f7478167d938824ce215eb3a906384e07e3bc74b3d98913e5ff0180
{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570",
        "puzzle_hash": "0x98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff18ff2fffff010180ffff01ff02ff36ffff04ff02ffff04ff05ffff04ff17ffff04ffff02ff26ffff04ff02ffff04ff0bff80808080ffff04ff2fffff04ff0bffff04ff5fff808080808080808080ffff01ff088080ff0180ffff04ffff01ffffffff4602ff3304ffff0101ff02ffff02ffff0

In [22]:
%%bash
opd ff02ffff01ff02ffff01ff02ffff03ffff18ff2fffff010180ffff01ff02ff36ffff04ff02ffff04ff05ffff04ff17ffff04ffff02ff26ffff04ff02ffff04ff0bff80808080ffff04ff2fffff04ff0bffff04ff5fff808080808080808080ffff01ff088080ff0180ffff04ffff01ffffffff4602ff3304ffff0101ff02ffff02ffff03ff05ffff01ff02ff5cffff04ff02ffff04ff0dffff04ffff0bff2cffff0bff24ff3880ffff0bff2cffff0bff2cffff0bff24ff3480ff0980ffff0bff2cff0bffff0bff24ff8080808080ff8080808080ffff010b80ff0180ff02ffff03ff0bffff01ff02ff32ffff04ff02ffff04ff05ffff04ff0bffff04ff17ffff04ffff02ff2affff04ff02ffff04ffff02ffff03ffff09ff23ff2880ffff0181b3ff8080ff0180ff80808080ff80808080808080ffff01ff02ffff03ff17ff80ffff01ff088080ff018080ff0180ffffffff0bffff0bff17ffff02ff3affff04ff02ffff04ff09ffff04ff2fffff04ffff02ff26ffff04ff02ffff04ff05ff80808080ff808080808080ff5f80ff0bff81bf80ff02ffff03ffff20ffff22ff4fff178080ffff01ff02ff7effff04ff02ffff04ff6fffff04ffff04ffff02ffff03ff4fffff01ff04ff23ffff04ffff02ff3affff04ff02ffff04ff09ffff04ff53ffff04ffff02ff26ffff04ff02ffff04ff05ff80808080ff808080808080ffff04ff81b3ff80808080ffff011380ff0180ffff02ff7cffff04ff02ffff04ff05ffff04ff1bffff04ffff21ff4fff1780ff80808080808080ff8080808080ffff01ff088080ff0180ffff04ffff09ffff18ff05ffff010180ffff010180ffff09ff05ffff01818f8080ff0bff2cffff0bff24ff3080ffff0bff2cffff0bff2cffff0bff24ff3480ff0580ffff0bff2cffff02ff5cffff04ff02ffff04ff07ffff04ffff0bff24ff2480ff8080808080ffff0bff24ff8080808080ffffff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff26ffff04ff02ffff04ff09ff80808080ffff02ff26ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff02ff5effff04ff02ffff04ff05ffff04ff0bffff04ffff02ff3affff04ff02ffff04ff09ffff04ff17ffff04ffff02ff26ffff04ff02ffff04ff05ff80808080ff808080808080ffff04ff17ffff04ff2fffff04ff5fffff04ff81bfff80808080808080808080ffff04ffff04ff20ffff04ff17ff808080ffff02ff7cffff04ff02ffff04ff05ffff04ffff02ff82017fffff04ffff04ffff04ff17ff2f80ffff04ffff04ff5fff81bf80ffff04ff0bff05808080ff8202ff8080ffff01ff80808080808080ffff02ff2effff04ff02ffff04ff05ffff04ff0bffff04ffff02ffff03ff3bffff01ff02ff22ffff04ff02ffff04ff05ffff04ff17ffff04ff13ffff04ff2bffff04ff5bffff04ff5fff808080808080808080ffff01ff02ffff03ffff09ff15ffff0bff13ff1dff2b8080ffff01ff0bff15ff17ff5f80ffff01ff088080ff018080ff0180ffff04ff17ffff04ff2fffff04ff5fffff04ff81bfffff04ff82017fff8080808080808080808080ff02ffff03ff05ffff011bffff010b80ff0180ff018080ffff04ffff01ffa024e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fdffa06a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff04ffff01ff02ffff01ff04ffff04ffff0133ffff04ff02ffff04ff05ff80808080ffff04ffff04ffff0148ffff04ff02ff808080ffff04ffff04ffff0149ffff04ff05ff808080ff80808080ffff06ff018080ff01808080

(a (q 2 (q 2 (i (logand 47 (q . 1)) (q 2 54 (c 2 (c 5 (c 23 (c (a 38 (c 2 (c 11 ()))) (c 47 (c 11 (c 95 ())))))))) (q 8)) 1) (c (q (((70 . 2) 51 . 4) (q . 1) 2 (a (i 5 (q 2 92 (c 2 (c 13 (c (sha256 44 (sha256 36 56) (sha256 44 (sha256 44 (sha256 36 52) 9) (sha256 44 11 (sha256 36 ())))) ())))) (q . 11)) 1) 2 (i 11 (q 2 50 (c 2 (c 5 (c 11 (c 23 (c (a 42 (c 2 (c (a (i (= 35 40) (q . -77) ()) 1) ()))) ())))))) (q 2 (i 23 () (q 8)) 1)) 1) (((sha256 (sha256 23 (a 58 (c 2 (c 9 (c 47 (c (a 38 (c 2 (c 5 ()))) ()))))) 95) 11 -65) 2 (i (not (all 79 23)) (q 2 126 (c 2 (c 111 (c (c (a (i 79 (q 4 35 (c (a 58 (c 2 (c 9 (c 83 (c (a 38 (c 2 (c 5 ()))) ()))))) (c -77 ()))) (q . 19)) 1) (a 124 (c 2 (c 5 (c 27 (c (any 79 23) ())))))) ())))) (q 8)) 1) (c (= (logand 5 (q . 1)) (q . 1)) (= 5 (q . -113))) 11 44 (sha256 36 48) (sha256 44 (sha256 44 (sha256 36 52) 5) (sha256 44 (a 92 (c 2 (c 7 (c (sha256 36 36) ())))) (sha256 36 ())))) ((a (i (l 5) (q 11 (q . 2) (a 38 (c 2 (c 9 ()))) (a 38 (c 2 (c 13 ())))) (q

In [24]:
%%bash
opd ffffa012d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bbaff8203ff80ff8203ffffffa088b33a14c0139a7a4221b118dfae4cdf1ee9343c30d6abed18ee614928b73118ff8203ffff018080

((0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba 1023) 1023 (0x88b33a14c0139a7a4221b118dfae4cdf1ee9343c30d6abed18ee614928b73118 1023 1))


In [25]:
%%bash
brun '(a (q 2 (q 2 (i (logand 47 (q . 1)) (q 2 54 (c 2 (c 5 (c 23 (c (a 38 (c 2 (c 11 ()))) (c 47 (c 11 (c 95 ())))))))) (q 8)) 1) (c (q (((70 . 2) 51 . 4) (q . 1) 2 (a (i 5 (q 2 92 (c 2 (c 13 (c (sha256 44 (sha256 36 56) (sha256 44 (sha256 44 (sha256 36 52) 9) (sha256 44 11 (sha256 36 ())))) ())))) (q . 11)) 1) 2 (i 11 (q 2 50 (c 2 (c 5 (c 11 (c 23 (c (a 42 (c 2 (c (a (i (= 35 40) (q . -77) ()) 1) ()))) ())))))) (q 2 (i 23 () (q 8)) 1)) 1) (((sha256 (sha256 23 (a 58 (c 2 (c 9 (c 47 (c (a 38 (c 2 (c 5 ()))) ()))))) 95) 11 -65) 2 (i (not (all 79 23)) (q 2 126 (c 2 (c 111 (c (c (a (i 79 (q 4 35 (c (a 58 (c 2 (c 9 (c 83 (c (a 38 (c 2 (c 5 ()))) ()))))) (c -77 ()))) (q . 19)) 1) (a 124 (c 2 (c 5 (c 27 (c (any 79 23) ())))))) ())))) (q 8)) 1) (c (= (logand 5 (q . 1)) (q . 1)) (= 5 (q . -113))) 11 44 (sha256 36 48) (sha256 44 (sha256 44 (sha256 36 52) 5) (sha256 44 (a 92 (c 2 (c 7 (c (sha256 36 36) ())))) (sha256 36 ())))) ((a (i (l 5) (q 11 (q . 2) (a 38 (c 2 (c 9 ()))) (a 38 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 2 94 (c 2 (c 5 (c 11 (c (a 58 (c 2 (c 9 (c 23 (c (a 38 (c 2 (c 5 ()))) ()))))) (c 23 (c 47 (c 95 (c -65 ()))))))))) (c (c 32 (c 23 ())) (a 124 (c 2 (c 5 (c (a 383 (c (c (c 23 47) (c (c 95 -65) (c 11 5))) 767)) (q ())))))) (a 46 (c 2 (c 5 (c 11 (c (a (i 59 (q 2 34 (c 2 (c 5 (c 23 (c 19 (c 43 (c 91 (c 95 ())))))))) (q 2 (i (= 21 (sha256 19 29 43)) (q 11 21 23 95) (q 8)) 1)) 1) (c 23 (c 47 (c 95 (c -65 (c 383 ())))))))))) 2 (i 5 (q . 27) (q . 11)) 1) 1)) (c (q 0x24e044101e57b3d8c908b8a38ad57848afd29d3eecc439dba45f4412df4954fd 0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) (c (q 2 (q 4 (c (q . 51) (c 2 (c 5 ()))) (c (c (q . 72) (c 2 ())) (c (c (q . 73) (c 5 ())) ()))) (r 1)) 1)))' '((0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba 1023) 1023 (0x88b33a14c0139a7a4221b118dfae4cdf1ee9343c30d6abed18ee614928b73118 1023 1))'

((70 0x814ad24f2503f67c7ae2efa8bbbf3e5e67481be3abe457c1b8893ad1d4733ad7) (51 0x0dc6eaf2fc4a44f6b97eece419d75622a4a3aeec5fa160105c8a4f2711339273 1023) (72 0x88b33a14c0139a7a4221b118dfae4cdf1ee9343c30d6abed18ee614928b73118) (73 1023))


In [21]:
%%bash
opd ffa088b33a14c0139a7a4221b118dfae4cdf1ee9343c30d6abed18ee614928b73118ff8203ffff0180

(0x88b33a14c0139a7a4221b118dfae4cdf1ee9343c30d6abed18ee614928b73118 1023 1)


In [7]:
from blspy import G2Element
spend_bundle = SpendBundle(
    [singleton_coin_spend],
    G2Element()
)
result = await network.push_tx(spend_bundle)
result

{'additions': [Coin(parent_coin_info=<bytes32: 319563d316c50eb4e8323a5a90c8db6a579802c694503f0232355b7270e5b4bc>, puzzle_hash=<bytes32: 98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: 98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a>, amount=1023)]}

In [30]:
from clvm_tools.binutils import disassemble
network.sim.pass_blocks(2)
singleton_coin_records = await network.sim_client.get_coin_records_by_parent_ids([launcher_id], include_spent_coins = True)
spent_coin_id = singleton_coin_records[0].coin.name()
spent_block_index = singleton_coin_records[0].spent_block_index
coin_spent = await network.sim_client.get_puzzle_and_solution(spent_coin_id, spent_block_index)
coin_spent
#solution = coin_spent[0].solution
#print(disassemble(solution))

CoinSpend(coin=Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: 98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a>, amount=1023), puzzle_reveal=SerializedProgram(ff02ffff01ff02ffff01ff02ffff03ffff18ff2fffff010180ffff01ff02ff36ffff04ff02ffff04ff05ffff04ff17ffff04ffff02ff26ffff04ff02ffff04ff0bff80808080ffff04ff2fffff04ff0bffff04ff5fff808080808080808080ffff01ff088080ff0180ffff04ffff01ffffffff4602ff3304ffff0101ff02ffff02ffff03ff05ffff01ff02ff5cffff04ff02ffff04ff0dffff04ffff0bff2cffff0bff24ff3880ffff0bff2cffff0bff2cffff0bff24ff3480ff0980ffff0bff2cff0bffff0bff24ff8080808080ff8080808080ffff010b80ff0180ff02ffff03ff0bffff01ff02ff32ffff04ff02ffff04ff05ffff04ff0bffff04ff17ffff04ffff02ff2affff04ff02ffff04ffff02ffff03ffff09ff23ff2880ffff0181b3ff8080ff0180ff80808080ff80808080808080ffff01ff02ffff03ff17ff80ffff01ff088080ff018080ff0180ffffffff0bffff0bff17ffff02ff3affff04ff02ffff04ff09ffff04ff2fffff04ffff02ff26ffff04f